In [1]:
import psycopg2

conexionDB = psycopg2.connect(
    host = "localhost",
    port = "5432",
    database = "libros",
    user = "postgres",
    password = "dan"
)
print("conexion exitosa")

cursor = conexionDB.cursor() # el cursor es el medio por el que envia y recibe los comandos sql

# tabla de autores
cursor.execute("""
    CREATE TABLE IF NOT EXISTS autores(
        id SERIAL PRIMARY KEY,
        nombre VARCHAR (100)
)
""")
# tabla de categorias
cursor.execute("""
    CREATE TABLE IF NOT EXISTS categorias(
        id SERIAL PRIMARY KEY,
        nombre VARCHAR (100)
)
""")
# tabla de libros
cursor.execute("""
    CREATE TABLE IF NOT EXISTS libros(
        id SERIAL PRIMARY KEY,
        titulo VARCHAR (500) NOT NULL,
        precio DECIMAL (10,2),
        rating INTEGER CHECK (rating >= 0 AND rating <= 5),
        categoria_id INTEGER,
        en_stock VARCHAR (100),
        url VARCHAR (1000),
        FOREIGN KEY (categoria_id) REFERENCES categorias (id)
        
)
""")
#titulo, precio, rating, categoria, en stock, fecha de publicacion, descripcion y url
# Tabla autores muchos <-> muchos
cursor.execute("""
    CREATE TABLE IF NOT EXISTS libros_autores(
        id SERIAL PRIMARY KEY,
        libro_id INTEGER,
        autor_id INTEGER,
        FOREIGN KEY (libro_id) REFERENCES libros(id) ON DELETE CASCADE,
        FOREIGN KEY (autor_id) REFERENCES autores(id) ON DELETE CASCADE,
        UNIQUE(libro_id, autor_id)
)
""")


conexionDB.commit()
print("Tabla creada exitosamente")

cursor.close()
conexionDB.close()

conexion exitosa
Tabla creada exitosamente


In [ ]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import threading
import random
import json

scrap = requests.session()

autores = [
    "Juan Pérez",
    "María García",
    "Carlos López", 
    "Ana Martínez", 
    "Luis Rodríguez", 
    "Sofia Fernández"
]

reating_convercion = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}

def scrapeaTodo(i):
    paginas = scrap.get(f"https://books.toscrape.com/catalogue/page-{i}.html")
    paginas.encoding = "utf-8"
    soup = BeautifulSoup(paginas.text,"html.parser")
    
    datos_libros = []

    for lib in soup.find_all("article", class_= "product_pod"):
        titulo = lib.h3.a["title"].strip()
        rating = reating_convercion[lib.find("p",class_="star-rating")["class"][-1]]
        precio = lib.find("p",class_="price_color").text
        stock = lib.find("p",class_="instock availability").text.strip()
        urlLibro = "https://books.toscrape.com/catalogue/" + lib.h3.a["href"]

        pagLibro = scrap.get(urlLibro)
        soup2 = BeautifulSoup(pagLibro.text,"html.parser")
        class_catego = soup2.find("ul",class_="breadcrumb")
        categoria = class_catego.find_all("li")[2].a.text.strip()

        datos_libros.append({
            "titulo":titulo,
            "precio":precio,
            "rating":rating,
            "categoria":categoria,
            "stock":stock,
            "urlLibro":urlLibro,
            "autor":random.choice(autores)
        })
    return datos_libros

todoslosLibros = []
bloquea = threading.Lock()

def scrapearPag(pagina):
    resultado = scrapeaTodo(pagina)
    with bloquea:
        todoslosLibros.extend(resultado)

with ThreadPoolExecutor(max_workers=5) as ejecutor:
    ejecutor.map(scrapearPag,range(1,51))



print("scrapeado los 1000 libros")